In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

model.config.problem_type = "single_label_classification"


model = model.to(device)

model.eval()

label_mapping = {0: 'neutral', 1: 'positive', 2: 'negative'}

print("Model loaded and ready for inferencing.")

d:\Work\miniconda_env\hf-pytorch\lib\site-packages\transformers\utils\hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded and ready for inferencing.


In [2]:
def predict_batch(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length = 128).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        pred = nn.functional.softmax(outputs.logits, dim=-1)
        predicted_labels = torch.argmax(pred, dim=-1)

    results = []
    for i, text in enumerate(texts):
        results.append({"text": text, 
                        "sentiment": label_mapping[predicted_labels[i].item()],
                        "confifence": pred[i][predicted_labels[i]].item()
                        })
        
    return results


In [3]:
sample_texts = [
	"I can't believe how bad the service was at that restaurant.", # Negative Sentiment
	"The movie was absolutely fantastic, I loved every minute of it!", # Positive Sentiment
	"It was okay, not great but not terrible either.", # Neutral Sentiment
	"This is the worst product I've ever bought!", # Strong Negative Sentiment
	"I am so thrilled with my new phone, it's amazing!", # Strong Positive Sentiment
	"The weather today is just perfect for a walk in the park.", # Positive Sentiment
	"I don't think this is going to work out well for us.", # Negative Sentiment
	"This book is a masterpiece, I couldn't put it down!", # Positive Sentiment
	"I really didn't enjoy the concert last night.", # Negative Sentiment
	"The food was decent, but I've had better.", # Neutral Sentiment
	"I just love waiting two hours for my coffee—it really makes my morning so special.", #Sarcasm with Positive Lexicon
	"It's not that the service was bad; it was just not completely unimpressive.", # Double Negation & Understatement
	"I adored the plot twists, but the pacing ruined what could've been an unforgettable read.", #Mixed Sentiment (Multipolarity)
	"Sure, the new update fixed half the bugs… but now it crashes every five minutes. Go figure.", #Idiomatic Expression
	"Great, my flight's delayed again—just what I needed before a red-carpet premiere.", # Irony Embedded in Domain Context
	"He really pulled a Gatsby by throwing that extravagant party—impressive, yet fundamentally hollow.", # Cultural Reference Ambiguity
	"wouldn't call their performance amazing; it was merely… adequate.", #Subtle Negation Scope
	#"Product arrived on time 😊 but looks nothing like the photos 😒.", #  Emoticon-Driven Tone Shift
	"Although the cinematography was breathtaking, the dialogue felt contrived and pretentious, leaving me torn.", # Complex Compound Sentence
	"The software's GUI is slick as hell, but the backend is a trainwreck—zero cohesion in the APIs." # Domain-Specific Jargon and Slang
]

results = predict_batch(sample_texts)

d:\Work\miniconda_env\hf-pytorch\lib\site-packages\torch\nn\modules\module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [4]:
for res in results:
    print(f"Text: {res['text']}")
    print(f"Predicted Sentiment: {res['sentiment']} (confidence: {res['confifence']:.2f})\n")

Text: I can't believe how bad the service was at that restaurant.
Predicted Sentiment: negative (confidence: 0.35)

Text: The movie was absolutely fantastic, I loved every minute of it!
Predicted Sentiment: negative (confidence: 0.35)

Text: It was okay, not great but not terrible either.
Predicted Sentiment: negative (confidence: 0.36)

Text: This is the worst product I've ever bought!
Predicted Sentiment: negative (confidence: 0.38)

Text: I am so thrilled with my new phone, it's amazing!
Predicted Sentiment: negative (confidence: 0.34)

Text: The weather today is just perfect for a walk in the park.
Predicted Sentiment: negative (confidence: 0.35)

Text: I don't think this is going to work out well for us.
Predicted Sentiment: negative (confidence: 0.37)

Text: This book is a masterpiece, I couldn't put it down!
Predicted Sentiment: negative (confidence: 0.36)

Text: I really didn't enjoy the concert last night.
Predicted Sentiment: negative (confidence: 0.36)

Text: The food was de